In [4]:
#import the necessities
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
from scipy.stats import bartlett, levene
from sklearn import linear_model
from sqlalchemy import create_engine
from statsmodels.tsa.stattools import acf
warnings.filterwarnings('ignore')

1. Interpretation and significance
Suppose that you would like to know how much families in the US are spending on recreation annually. Use the following estimated model:

        𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒 = 873 + 0.0012𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒 + 0.00002𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒2 − 223.57ℎ𝑎𝑣𝑒_𝑘𝑖𝑑𝑠
        
Here, expenditure is the annual spending on recreation in US dollars, annual_income is the annual income in US dollars, and have_kids is a dummy variable indicating the families with children. Interpret the estimated coefficients. What additional statistics can be given in order to ensure that your interpretations make sense statistically? Write up your answer.

873 is the intercept or bias. We don't have any p values or know what annual_income2 is, so we can't determine if the differences are significant. Assuming that each of these coefficients are indeed significant, we can state that increasing the annual_income by 1 increases expenditure by 0.0012, or stated differently, you have to increase annual_income by 833.33 to increase expenditure by 1. We can also state that having kids decreases expenditure by 223.57. However, since it's a dummy variable, it might be better to view it in the context of other variables.

2. Weather model
First, load the dataset from the weatherinszeged table from Thinkful's database.

In [5]:
#create and dispose of connection
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
weather_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, weather_db))

weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

engine.dispose()

Build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using OLS. Are the estimated coefficients statistically significant? Are the signs of the estimated coefficients in line with your previous expectations? Interpret the estimated coefficients. What are the relations between the target and the explanatory variables?

In [6]:
#I always like creating a copy of the df so I don't have to reestablish a connection
weather_df_copy = weather_df.copy()

In [7]:
weather_df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [9]:
#we'll create the target and the feature df
weather_y = (weather_df['apparenttemperature'] - weather_df['temperature'])
weather_X = weather_df[['humidity', 'windspeed']]
weather_X = sm.add_constant(weather_X)

In [12]:
#build a linear regression
weather_lrm = linear_model.LinearRegression()
weather_lrm.fit(weather_X, weather_y)
print('Coefficients: \n', weather_lrm.coef_)
print('\nIntercept: \n', weather_lrm.intercept_)

Coefficients: 
 [ 0.         -3.02918594 -0.11929075]

Intercept: 
 2.43810541518771


In [13]:
#do the OLS stats
weather_results = sm.OLS(weather_y, weather_X).fit()
display(weather_results.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Wed, 13 Jan 2021   Prob (F-statistic):               0.00
Time:                        17:22:02   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The estimated coefficients are likely significant, as the p values are essentially zero. From personal experience, humidity seems to make the temperature feel hotter, so seeing a negative here is surprising. I did expect it for windspeed but expected it to be a larger coefficient.
The relations are as follows: a 1 point increase in humidity is a 3.03 decrease in the difference between apparent temperature and temperature and a 1 point increase in windspeed is a 0.12 decrease in the difference between apparent temperature and temperature.

Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Are the coefficients statistically significant? Did the signs of the estimated coefficients for humidity and windspeed change? Interpret the estimated coefficients.

In [14]:
#create the column that includes the interaction and create the OLS
weather_df['humidity-windspeed interaction'] = (weather_df['humidity'] * weather_df['windspeed'])
weather_i_y = (weather_df['apparenttemperature'] - weather_df['temperature'])
weather_i_X = weather_df[['humidity', 'windspeed', 'humidity-windspeed interaction']]
weather_i_X = sm.add_constant(weather_i_X)

weather_i_result = sm.OLS(weather_i_y, weather_i_X).fit()

display(weather_i_result.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Wed, 13 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:00:48   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                              0.0839      0.033      2.511      0.012       0.018       0.149
humidity                           0.1775      0.043      4.133      0.000       0.093       0.262
windspeed                          0.0905      0.002     36.797      0.000       0.086       0.095
humidity-windspeed interaction    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The estimated coefficients are likely significant, as the p values are essentially zero.
The signs of the coefficients did change from the previous model.
The relations are as follows: a 1 point increase in humidity is a 0.18 increase in the difference between apparent temperature and temperature, a 1 point increase in windspeed is a 0.09 decrease in the difference between apparent temperature and temperature, and a 1 point increase in the interaction between humidity and windspeed is a 0.30 decrease in the difference between apparent temperature and temperature.

3. House prices model
Load the houseprices data from Thinkful's database.

In [19]:
#create and dispose of connection
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
house_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, house_db))

houses_df = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

In [20]:
#I always make a copy of the df
houses_copy_df = houses_df.copy()

Run your house prices model again and interpret the results. Which features are statistically significant, and which are not?

In [24]:
#create the variables needed
#previously used variables were lotarea, overallqual, overallcond, centralair 
houses_y = houses_df['saleprice']
dummy_df = pd.get_dummies(houses_df['centralair'], drop_first=True)
houses_X = houses_df[['lotarea', 'overallqual', 'overallcond']].merge(
    dummy_df, left_index=True, right_index=True)
#and add a constant
houses_X = sm.add_constant(houses_X)

In [29]:
house_results = sm.OLS(houses_y, houses_X).fit()
display(house_results.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.659
Method:                 Least Squares   F-statistic:                     705.2
Date:                Wed, 13 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:33:40   Log-Likelihood:                -17757.
No. Observations:                1460   AIC:                         3.552e+04
Df Residuals:                    1455   BIC:                         3.555e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.074e+05   8919.167    -12.038      0.000   -1.25e+05   -8.99e+04
lotarea         1.4443      0.122     11.796      0.000       1.204       1.684
overallqual   4.37e+04    924.907     47.248      0.000    4.19e+04    4.55e+04
overallcond  -803.6904   1108.972     -0.725      0.469   -2979.045    1371.664
Y            1.181e+04   5176.863      2.281      0.023    1655.524     2.2e+04
==============================================================================
Omnibus:                      591.848   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7641.681
Skew:                           1.522   Prob(JB):                         0.00
Kurtosis:                      13.787   Cond. No.                     1.09e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.09e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Based on the p value, overallcond is not significant, so we'll drop it and run the test again.

Now, exclude the insignificant features from your model. Did anything change?

In [40]:
#rerunning the test without overallcond
houses_2_y = houses_df['saleprice']
houses_2_X = houses_df[['lotarea', 'overallqual']].merge(
    dummy_df, left_index=True, right_index=True)
#and add a constant
houses_2_X = sm.add_constant(houses_2_X)

In [41]:
house_2_results = sm.OLS(houses_2_y, houses_2_X).fit()
display(house_2_results.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.659
Method:                 Least Squares   F-statistic:                     940.5
Date:                Wed, 13 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:39:26   Log-Likelihood:                -17757.
No. Observations:                1460   AIC:                         3.552e+04
Df Residuals:                    1456   BIC:                         3.554e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.119e+05   6424.464    -17.411      0.000   -1.24e+05   -9.93e+04
lotarea         1.4442      0.122     11.798      0.000       1.204       1.684
overallqual  4.379e+04    916.959     47.753      0.000     4.2e+04    4.56e+04
Y            1.125e+04   5117.296      2.198      0.028    1208.844    2.13e+04
==============================================================================
Omnibus:                      591.663   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7628.923
Skew:                           1.522   Prob(JB):                         0.00
Kurtosis:                      13.777   Cond. No.                     8.62e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.62e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The three variables (lotarea, overallqual, and centralair) are all statistically significant and their coefficients have changed from the previous iteration.

Interpret the statistically significant coefficients by quantifying their relations with the house prices. Which features have a more prominent effect on house prices?

The relations are as follows: a 1 point increase in lotarea is a 1.444 ncrease in saleprice, a 1 point increase in overallqual is a 43,790 increase in saleprice, and having centralair is a 11,250 increase relative to not having centralair. Given that the lotarea is measured in increaments of 1,000, it would probably be better stated that given a 1,000 point increase in lotarea is a 1,444 increase in saleprice.
Of the ones tested here, overallcond has the highest effect on the saleprice.

Do the results sound reasonable to you? If not, try to explain the potential reasons.

Mostly reasonable. I expected centralair to have a more prominent effect than it did, but that could be explained in a couple of ways. It could be that centralair is factored into overallqual, or it could be that the cost of installing centralair isn't much higher than 11,250, meaning that it isn't as much of a value as I expected, or that houses outside of my area (Southern US) doesn't need centralair as much as we do in the Southern US, or that people are willing to use other solutions than centralair to cool their houses.